# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from transformers import DataCollatorWithPadding
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from peft import get_peft_model
import numpy as np
from datasets import load_dataset


In [2]:
# https://huggingface.co/docs/transformers/training
# or load the separate splits if the dataset has train/validation/test splits
train_dataset = load_dataset("cmotions/NL_restaurant_reviews", split="train")
valid_dataset = load_dataset("cmotions/NL_restaurant_reviews", split="validation")
test_dataset  = load_dataset("cmotions/NL_restaurant_reviews", split="test")

#look at full dataset
dataset = load_dataset("cmotions/NL_restaurant_reviews")
print(dataset)

#looks at characteristics of train
print(train_dataset)

# Inspect the first example. See what a restaurant review looks like
print(train_dataset[0])

#length of dataset
print(len(train_dataset))

DatasetDict({
    train: Dataset({
        features: ['restaurant_ID', 'restaurant_review_ID', 'michelin_label', 'score_total', 'score_food', 'score_service', 'score_decor', 'fame_reviewer', 'reviewscore_food', 'reviewscore_service', 'reviewscore_ambiance', 'reviewscore_waiting', 'reviewscore_value', 'reviewscore_noise', 'review_text', 'review_length'],
        num_rows: 116693
    })
    validation: Dataset({
        features: ['restaurant_ID', 'restaurant_review_ID', 'michelin_label', 'score_total', 'score_food', 'score_service', 'score_decor', 'fame_reviewer', 'reviewscore_food', 'reviewscore_service', 'reviewscore_ambiance', 'reviewscore_waiting', 'reviewscore_value', 'reviewscore_noise', 'review_text', 'review_length'],
        num_rows: 14587
    })
    test: Dataset({
        features: ['restaurant_ID', 'restaurant_review_ID', 'michelin_label', 'score_total', 'score_food', 'score_service', 'score_decor', 'fame_reviewer', 'reviewscore_food', 'reviewscore_service', 'reviewscore_am

In [3]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

splits = ["train", "validation", "test"]

tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["review_text"], truncation=True), batched=True
    )
    
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "negative review", 1: "positive review"},
    label2id={"negative review": 0, "positive review": 1},
)

# Unfreezes all model layers, making them trainable 
for param in model.parameters():
    param.requires_grad = True
    
    
print(model)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [4]:
# Saving the model
model.save("/tmp/your_model_name")

AttributeError: 'DistilBertForSequenceClassification' object has no attribute 'save'

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.